# Day 1 - Examples: Summarization

In this notebook we offer an example of text summarization. That is, given a large document, we ask the model to return a 1 sentence summary. 

References/Further Reading:

* PaLM: More elaborate prompts can be found at https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_summarization.ipynb

In [58]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv("../../.env")

True

## Setup template

We setup a template here asking the model to summarize a text from the given document into a single sentence. We can then replace different documents into this template and acquire summaries each time.

In [9]:
document = '<document>'
template_prompt=f'''Summarize this in 1 sentence. \nDocument: \"\"\"{document}\"\"\"\n'''
print(template_prompt)

Summarize this in 1 sentence. 
Document: """<document>"""



#### User Input

Here we add in the document we want the model to read. Feel free to modify this to a document of your choice!

In [10]:
select_document = """
Overview:
Tech Solutions Inc. is a leading technology consulting firm specializing in providing innovative solutions to businesses across various industries. We offer a comprehensive range of services including software development, IT consulting, project management, and cybersecurity solutions. 
With a strong focus on delivering exceptional quality and customer satisfaction, we have established ourselves as a trusted partner for organizations seeking digital transformation.
They were founded on April 12, 2005 and raise their first seed in 2007 and series A on May 25, 2007.

Founders:

Background: John Smith is a visionary entrepreneur with over 20 years of experience in the technology industry. He has a deep understanding of market trends and has successfully led several software development projects for multinational corporations.
Role in the Company: As a co-founder of Tech Solutions Inc., John Smith plays a pivotal role in shaping the company's strategic direction. His expertise in software development and leadership skills have been instrumental in driving the company's growth.
Sarah Johnson:

Background: Sarah Johnson is a highly accomplished technologist with a strong background in software engineering. She has extensive experience in managing complex IT projects and has a proven track record of delivering innovative solutions.
Role in the Company: As a co-founder of Tech Solutions Inc., Sarah Johnson leads the company's technical operations. Her deep knowledge of software engineering principles and commitment to excellence have been crucial in establishing the company as a leader in the industry.
Together, John Smith and Sarah Johnson founded Tech Solutions Inc. with the aim of providing cutting-edge technology solutions to help businesses thrive in the digital age. Their combined expertise and passion for innovation have been instrumental in the company's success.
"""


## OpenAI

Summarizing text with OpenAI models is pretty simple. We pretty much just send the template (with our document added in) to the API and the output will be the model's attempt at a summary!

In [4]:
import openai

prompt = template_prompt.replace('<document>',select_document)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

chatbot_response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages,
  temperature=0,
  max_tokens=1500,
)

print(chatbot_response.choices[0].message["content"])

Tech Solutions Inc., founded by John Smith and Sarah Johnson, is a leading technology consulting firm offering a range of services including software development, IT consulting, project management, and cybersecurity solutions, with a focus on exceptional quality and customer satisfaction.


## Google

Summarizing text with PaLM is also trivial. We essentially do the same thing here in that we take our prompt and send it to the model.

In [5]:
from vertexai.preview.language_models import TextGenerationModel

prompt = template_prompt.replace('<document>',select_document)

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(prompt, max_output_tokens=1024)
print(response.text)

Tech Solutions Inc. is a leading technology consulting firm founded by John Smith and Sarah Johnson in 2005.


## Open Source LLM: Falcon-7B

In [11]:
import requests
import os

prompt = template_prompt.replace('<document>',select_document)

headers = {"Authorization": f"Bearer {os.environ.get('HUGGINGFACEHUB_API_TOKEN')}"}

def query(payload):
    response = requests.request("POST", os.environ.get("HUGGINGFACEHUB_ENDPOINT"), headers=headers, json=payload)
    return response.json()

data = query({"inputs": prompt, "parameters": {"max_new_tokens": 250, "return_full_text": False}})

print(data[0]['generated_text'])

Tech Solutions Inc. was founded on April 12, 2005, by John Smith and Sarah Johnson. They specialize in providing innovative solutions to businesses across various industries. Their strong focus on delivering exceptional quality and customer satisfaction has established them as a trusted partner for organizations seeking digital transformation.


# Prompt Chaining

Prompt Chaining is a prompting technique that involves using the output of one prompt as a part of or as the whole input to the next prompt. 
This can help us use an LLM for larger bodies of text that may be too long for the model to work effectively with. You can read more about Prompt Chaining [here](https://txt.cohere.com/generative-ai-part-5/).

Lets perform CoT reasoning on the same summarization problem from above. Note that we only show this with a single API. However, you should be able to use this with any of the other APIs too as we only change the prompt itself. 

We also make a modification to the input here - we use a much larger document! We then proceed to chunk this document into a number of smaller segments, obtain a summary for each of these segments, then join these summaries together and get an overall summary!

In [59]:
document = '<document>'
template_prompt=f'''Summarize this in 1 sentence. \nDocument: \"\"\"{document}\"\"\"\n'''
print(template_prompt)

Summarize this in 1 sentence. 
Document: """<document>"""



In [60]:
# Arbitrarily picked sections of the Wikipedia page for Canada
document = """
Introduction
Canada is a country in North America. Its ten provinces and three territories extend from the Atlantic Ocean to the Pacific Ocean and northward into the Arctic Ocean, making it the world's second-largest country by total area, with the world's longest coastline. Its border with the United States is the world's longest international land border. The country is characterized by a wide range of both meteorologic and geological regions. It is sparsely inhabited, with the vast majority residing south of the 55th parallel in urban areas. Canada's capital is Ottawa and its three largest metropolitan areas are Toronto, Montreal, and Vancouver.

Indigenous peoples have continuously inhabited what is now Canada for thousands of years. Beginning in the 16th century, British and French expeditions explored and later settled along the Atlantic coast. As a consequence of various armed conflicts, France ceded nearly all of its colonies in North America in 1763. In 1867, with the union of three British North American colonies through Confederation, Canada was formed as a federal dominion of four provinces. This began an accretion of provinces and territories and a process of increasing autonomy from the United Kingdom, highlighted by the Statute of Westminster, 1931, and culminating in the Canada Act 1982, which severed the vestiges of legal dependence on the Parliament of the United Kingdom.

Canada is a parliamentary liberal democracy and a constitutional monarchy in the Westminster tradition. The country's head of government is the prime minister, who holds office by virtue of their ability to command the confidence of the elected House of Commons and is "called upon" by the governor general, representing the monarch of Canada, the ceremonial head of state. The country is a Commonwealth realm and is officially bilingual (English and French) in the federal jurisdiction. It is very highly ranked in international measurements of government transparency, quality of life, economic competitiveness, innovation and education. It is one of the world's most ethnically diverse and multicultural nations, the product of large-scale immigration. Canada's long and complex relationship with the United States has had a significant impact on its history, economy, and culture.

A developed country, Canada has one of the highest nominal per capita income globally and its advanced economy ranks among the largest in the world, relying chiefly upon its abundant natural resources and well-developed international trade networks. Canada is recognized as a middle power for its role in international affairs, with a tendency to pursue multilateral solutions. Canada's peacekeeping role during the 20th century has had a significant influence on its global image. Canada is part of multiple major international and intergovernmental institutions.

<Section>: Etymology
While a variety of theories have been postulated for the etymological origins of Canada, the name is now accepted as coming from the St. Lawrence Iroquoian word kanata, meaning "village" or "settlement". In 1535, Indigenous inhabitants of the present-day Quebec City region used the word to direct French explorer Jacques Cartier to the village of Stadacona. Cartier later used the word Canada to refer not only to that particular village but to the entire area subject to Donnacona (the chief at Stadacona); by 1545, European books and maps had begun referring to this small region along the Saint Lawrence River as Canada.

From the 16th to the early 18th century, "Canada" referred to the part of New France that lay along the Saint Lawrence River. In 1791, the area became two British colonies called Upper Canada and Lower Canada. These two colonies were collectively named the Canadas until their union as the British Province of Canada in 1841.

Upon Confederation in 1867, Canada was adopted as the legal name for the new country at the London Conference and the word Dominion was conferred as the country's title. By the 1950s, the term Dominion of Canada was no longer used by the United Kingdom, which considered Canada a "Realm of the Commonwealth".

The Canada Act 1982, which brought the constitution of Canada fully under Canadian control, referred only to Canada. Later that year, the name of the national holiday was changed from Dominion Day to Canada Day. The term Dominion was used to distinguish the federal government from the provinces, though after the Second World War the term federal had replaced dominion.

<Section>: Geography
By total area (including its waters), Canada is the second-largest country in the world, after Russia. By land area alone, Canada ranks fourth, due to having the world's largest area of fresh water lakes. Stretching from the Atlantic Ocean in the east, along the Arctic Ocean to the north, and to the Pacific Ocean in the west, the country encompasses 9,984,670 km2 (3,855,100 sq mi) of territory. Canada also has vast maritime terrain, with the world's longest coastline of 243,042 kilometres (151,019 mi). In addition to sharing the world's largest land border with the United States—spanning 8,891 km (5,525 mi)—Canada shares a land border with Greenland (and hence the Kingdom of Denmark) to the northeast, on Hans Island, and a maritime boundary with France's overseas collectivity of Saint Pierre and Miquelon to the southeast. Canada is also home to the world's northernmost settlement, Canadian Forces Station Alert, on the northern tip of Ellesmere Island—latitude 82.5°N—which lies 817 kilometres (508 mi) from the North Pole.

Canada can be divided into seven physiographic regions: the Canadian Shield, the interior plains, the Great Lakes-St. Lawrence Lowlands, the Appalachian region, the Western Cordillera, Hudson Bay Lowlands, and the Arctic Archipelago. Boreal forests prevail throughout the country, ice is prominent in northern Arctic regions and through the Rocky Mountains, and the relatively flat Canadian Prairies in the southwest facilitate productive agriculture. The Great Lakes feed the St. Lawrence River (in the southeast) where the lowlands host much of Canada's economic output. Canada has over 2,000,000 lakes—563 of which are larger than 100 km2 (39 sq mi)—containing much of the world's fresh water. There are also fresh-water glaciers in the Canadian Rockies, the Coast Mountains, and the Arctic Cordillera. Canada is geologically active, having many earthquakes and potentially active volcanoes, notably Mount Meager massif, Mount Garibaldi, Mount Cayley, and the Mount Edziza volcanic complex.

<Section>: Climate
Average winter and summer high temperatures across Canada vary from region to region. Winters can be harsh in many parts of the country, particularly in the interior and Prairie provinces, which experience a continental climate, where daily average temperatures are near −15 °C (5 °F), but can drop below −40 °C (−40 °F) with severe wind chills. In non-coastal regions, snow can cover the ground for almost six months of the year, while in parts of the north snow can persist year-round. Coastal British Columbia has a temperate climate, with a mild and rainy winter. On the east and west coasts, average high temperatures are generally in the low 20s °C (70s °F), while between the coasts, the average summer high temperature ranges from 25 to 30 °C (77 to 86 °F), with temperatures in some interior locations occasionally exceeding 40 °C (104 °F).

Much of Northern Canada is covered by ice and permafrost. The future of the permafrost is uncertain because the Arctic has been warming at three times the global average as a result of climate change in Canada. Canada's annual average temperature over land has risen by 1.7 °C (3.1 °F), with changes ranging from 1.1 to 2.3 °C (2.0 to 4.1 °F) in various regions, since 1948. The rate of warming has been higher across the North and in the Prairies. In the southern regions of Canada, air pollution from both Canada and the United States—caused by metal smelting, burning coal to power utilities, and vehicle emissions—has resulted in acid rain, which has severely impacted waterways, forest growth, and agricultural productivity in Canada.

<Section>: Biodiversity
Canada is divided into 15 terrestrial and five marine ecozones. These ecozones encompass over 80,000 classified species of Canadian wildlife, with an equal number yet to be formally recognized or discovered. Although Canada has a low percentage of endemic species compared to other countries, due to human activities, invasive species, and environmental issues in the country, there are currently more than 800 species at risk of being lost. About 65 percent of Canada's resident species are considered "Secure". Over half of Canada's landscape is intact and relatively free of human development. The boreal forest of Canada is considered to be the largest intact forest on Earth, with approximately 3,000,000 km2 (1,200,000 sq mi) undisturbed by roads, cities or industry. Since the end of the last glacial period, Canada has consisted of eight distinct forest regions, with 42 percent of its land area covered by forests (approximately 8 percent of the world's forested land).

Approximately 12.1 percent of the nation's landmass and freshwater are conservation areas, including 11.4 percent designated as protected areas. Approximately 13.8 percent of its territorial waters are conserved, including 8.9 percent designated as protected areas. Canada's first National Park, Banff National Park established in 1885, spans 6,641 square kilometres (2,564 sq mi) of mountainous terrain, with many glaciers and ice fields, dense coniferous forest, and alpine landscapes. Canada's oldest provincial park, Algonquin Provincial Park, established in 1893, covers an area of 7,653.45 square kilometres (2,955.01 sq mi). It is dominated by old-growth forest with over 2,400 lakes and 1,200 kilometres of streams and rivers. Lake Superior National Marine Conservation Area is the world's largest freshwater protected area, spanning roughly 10,000 square kilometres (3,900 sq mi) of lakebed, its overlaying freshwater, and associated shoreline on 60 square kilometres (23 sq mi) of islands and mainland. Canada's largest national wildlife region is the Scott Islands Marine National Wildlife Area, which spans 11,570.65 square kilometres (4,467.45 sq mi) and protects critical breeding and nesting habitat for over 40 percent of British Columbia's seabirds. Canada's 18 UNESCO Biosphere Reserves cover a total area of 235,000 square kilometres (91,000 sq mi).
"""

In [61]:
import openai

def get_chunks(document):
    # We just use a manual split here for test purposes
    return document.split("<Section>: ")

def summarize(template_prompt, document):
    prompt = template_prompt.replace('<document>', document)

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ]

    chatbot_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0,
    max_tokens=1500,
    )

    return chatbot_response.choices[0].message["content"]


In [62]:
# Summarize each chunk
summaries = []
for chunk in get_chunks(document):
    summaries.append(summarize(template_prompt, chunk))

In [63]:
# Pass the chunk summaries for a final summary
final_summary = summarize(template_prompt, '\n'.join(summaries))

In [64]:
from pprint import pprint
pprint(final_summary)

('Canada is a large, diverse country with a developed economy heavily reliant '
 'on natural resources and international trade, and over half of its landscape '
 'is intact and relatively free of human development.')


### Summaries Acquired!

We now have a list of summaries created by our model. Next, we combine them into a single document and ask our model to summarize this. This chaining of output into input is known as prompt chaining.

### Removing Personal Identifiable Information (PII) with PrivateAI

We may not always want to send our documents to an outside API due to concerns about sensitive data. Thus anonymizing this data is an important aspect to consider. PrivateAI uses machine learning to identify such sensitive data such as names, locations, insurance numbers etc. and replace them with generic labels. It is primarily designed to be self-hosted by the user so that no data is leaked outside your environment. For the purposes of this example, we just use their public API.

Although we use this before getting the final summary for example purposes, in practice you would do this with every model call to prevent leaking of sensitive data.

In [65]:
import requests
import os

def remove_pii(text):
    url = "https://api.private-ai.com/deid/v3/process/text"
    payload = {"text": [text]}
    headers = {"Content-Type": "application/json", "x-api-key": os.environ.get("PRIVATE_AI_API_KEY")}
    response = requests.post(url, json=payload, headers=headers)
    return response.json()[0]['processed_text']

In [66]:
pprint(summarize(template_prompt, remove_pii(final_summary)))

('[LOCATION_COUNTRY_1] is a large and diverse country with a developed economy '
 'heavily reliant on natural resources and international trade, and over half '
 'of its landscape is intact and relatively free of human development.')
